<a href="https://colab.research.google.com/github/data-88/53c/blob/main/Anotec_API_beta.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# install requirements 
!pip install flask
!pip install pyngrok
!pip install google-auth
!pip install flask pandas


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
# generate the synthetic data

import pandas as pd
import random
from datetime import datetime
from pyngrok import ngrok
from sklearn.ensemble import IsolationForest

# Set ngrok authtoken
ngrok.set_auth_token('2Qjzc8geYG4L4kLh33kyZxyTUYc_rfbxTEokajugbd4iGfrX')

def generate_user_id():
    user_ids = ['1234', '5678', '9012', '3456', '7890']
    return random.choice(user_ids)

# Generate random transaction amount
def generate_amount():
    return round(random.uniform(0, 1000), 2)

# Generate random date within the last year
def generate_date():
    end_date = datetime.now()
    start_date = end_date.replace(year=end_date.year - 1)
    random_date = start_date + (end_date - start_date) * random.random()
    return random_date.strftime('%Y-%m-%d')

# Generate random IP address
def generate_ip():
    ip_parts = [str(random.randint(0, 255)) for _ in range(4)]
    return '.'.join(ip_parts)

# Generate transactions
transactions = []
num_transactions = 1000

for _ in range(num_transactions):
    transaction_id = str(random.randint(1001, 11000))
    user_id = generate_user_id()
    amount = generate_amount()
    date = generate_date()
    ip = generate_ip()

    transactions.append({
        'Transaction ID': transaction_id,
        'User ID': user_id,
        'Amount': amount,
        'Date': date,
        'IP Address': ip
    })

# Convert transactions to DataFrame
transactions_df = pd.DataFrame(transactions)

# Save transactions to CSV
transactions_df.to_csv('transactions.csv', index=False)


In [ ]:
import pandas as pd
import threading
from sklearn.ensemble import IsolationForest
from flask import Flask, request, jsonify, render_template  
from pyngrok import ngrok

app = Flask(__name__)

# Train the Isolation Forest model
clf = IsolationForest(contamination=0.1, random_state=42)
clf.fit(transactions_df[['User ID', 'Transaction ID', 'Amount']])

# Initialize DataFrame to store anomalies
anomalies_df = pd.DataFrame(columns=['Transaction ID', 'User ID', 'Amount'])

@app.route('/')
def display_anomalies():
    # Read anomalies data from CSV
    anomalies_df = pd.read_csv('anomalies.csv')

    # Convert anomalies data to a list of dictionaries
    anomalies_list = anomalies_df.to_dict('records')

    return render_template('anomalies.html', anomalies=anomalies_list)

    
@app.route('/check-anomaly', methods=['POST'])
def check_anomaly():
    data = request.json

    # Extract the transaction ID, user ID, and amount from the input data
    transaction_id = data['Transaction ID']
    user_id = data['User ID']
    amount = data['Amount']

    # Create a DataFrame with the input data
    input_df = pd.DataFrame({'User ID': [user_id], 'Transaction ID': [transaction_id], 'Amount': [amount]})

    # Use the trained Isolation Forest model to predict anomalies
    is_anomaly = clf.predict(input_df) == -1

    # Append anomaly to DataFrame if detected
    if is_anomaly[0]:
        anomalies_df.loc[len(anomalies_df)] = [transaction_id, user_id, amount]

        # Save anomalies DataFrame to CSV file
        anomalies_df.to_csv('anomalies.csv', index=False)

    response = {
        'is_anomaly': bool(is_anomaly[0])
    }

    return jsonify(response)

# Start the Flask app in a new thread
threading.Thread(target=app.run, kwargs={"host": "0.0.0.0", "port": 5000, "use_reloader": False}).start()

# Open an ngrok tunnel to the Flask app
public_url = ngrok.connect(5000).public_url
print(" * ngrok tunnel \"{}\" -> \"http://127.0.0.1:{}\"".format(public_url, 5000))

# Update any base URLs to use the public ngrok URL
app.config["BASE_URL"] = public_url

# ... Update inbound traffic via APIs to use the public-facing ngrok URL

# Keep the main thread alive to keep the ngrok tunnel open
while True:
    pass


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but IsolationForest was fitted with feature names
  warnings.warn(


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit


 * ngrok tunnel "https://e613-35-237-76-99.ngrok-free.app" -> "http://127.0.0.1:5000"


INFO:werkzeug:127.0.0.1 - - [04/Jun/2023 15:12:32] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [04/Jun/2023 15:12:34] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [04/Jun/2023 15:15:18] "POST /check-anomaly HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [04/Jun/2023 15:15:23] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [04/Jun/2023 15:16:03] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [04/Jun/2023 15:16:38] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [04/Jun/2023 15:18:15] "POST /check-anomaly HTTP/1.1" 400 -
INFO:werkzeug:127.0.0.1 - - [04/Jun/2023 15:18:25] "POST /check-anomaly HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [04/Jun/2023 15:18:33] "POST /check-anomaly HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [04/Jun/2023 15:18:37] "POST /check-anomaly HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [04/Jun/2023 15:18:40] "POST /check-anomaly HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [04/Jun/2023 15:18:49] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127

In [ ]:
pwd